In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os, sys
%matplotlib inline

os.chdir('..\..\..\..')

from input_configuration import base_year

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
working_dir = r'outputs/agg/dash'
pd.options.display.float_format = '{:0,.2f}'.format
format_percent = "{:,.2%}".format

In [3]:
def load_data(fname):
    """ open and join model and survey data """

    df_model = pd.read_csv(os.path.join(working_dir,fname+'.csv'))
    df_model['source'] = 'model'
    df_survey = pd.read_csv(os.path.join(working_dir,'survey',fname+'.csv'))
    df_survey['source'] = 'survey'
    df = df_model.append(df_survey)

    return df

In [4]:
parcel_geog = pd.read_sql_table('parcel_'+base_year+'_geography', 'sqlite:///inputs/db/soundcast_inputs.db',
        columns=['CountyName','ParcelID', 'subarea_flag'])

In [5]:
def dist_by_mode(purpose=None, county_filter='', neg_county_filter=False):
    df = load_data('tour_distance')
    if county_filter!='' :
        if neg_county_filter:
            df = df.loc[df.person_subarea!=county_filter]
        else:
            df = df.loc[df.person_subarea==county_filter]            
    # Trim any unreasonable survey distances off by taking max distance as max modeled bin value
    max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
    df = df.loc[df['tautodist_bin'] <= max_model_dist]
    df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
    df_dist = pd.DataFrame()
    for source in ['model','survey']:
        if purpose is not None:
            _df = df[(df['source'] == source) & (df['pdpurp'] == purpose)]
        else:
            _df = df[(df['source'] == source)]
        df1 = _df.groupby('tmodetp').sum()[['wt_autodist']].reset_index()
        df2 = _df.groupby('tmodetp').sum()[['toexpfac']].reset_index ()
        _df = df1.merge(df2, on='tmodetp')
        _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
        _df['source'] = source
        df_dist = df_dist.append(_df)
    df = df_dist.pivot_table(index='tmodetp', columns='source', aggfunc='first', values='commute_distance')
    df.rename(columns={'tmodetp': 'Tour Mode'}, inplace=True)
    for col in ['School Bus',' Park']:
        if (col in df.index) & (purpose != 'School'):
            df.drop(col, axis=0, inplace=True)
        if (col != 'School Bus') & (col in df.index):
            df.drop(col, axis=0, inplace=True)
    
    return df

### Total

In [6]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.71
survey,4.28


## City of SeaTac

In [7]:
df = load_data('tour_distance')
df = df.loc[df['person_subarea']==1]
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,4.01
survey,3.45


## Other

In [8]:
df = load_data('tour_distance')
df = df.loc[df['person_subarea']!=1]
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.70
survey,4.29


### By Purpose

In [9]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True)
df

source,model,survey
pdpurp,,
Escort,7.11,3.88
Meal,5.45,4.22
Personal Business,6.82,5.49
School,3.71,4.28
Shop,4.80,4.53
Social,4.67,4.89
Work,12.06,11.51


## City of SeaTac

In [10]:
df = load_data('tour_distance')
df = df.loc[df['person_subarea']==1]
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True, errors='ignore')
df

source,model,survey
pdpurp,,
Escort,6.92,14.50
Meal,6.05,2.64
Personal Business,6.96,6.29
School,4.01,3.45
Shop,4.87,2.68
Social,5.21,5.06
Work,10.16,8.96


## Other

In [11]:
df = load_data('tour_distance')
df = df.loc[df['person_subarea']!=1]
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True, errors='ignore')
df

source,model,survey
pdpurp,,
Escort,7.12,3.84
Meal,5.43,4.23
Personal Business,6.81,5.48
School,3.70,4.29
Shop,4.80,4.55
Social,4.66,4.89
Work,12.10,11.54


# Tour Distance
### All Tours

In [12]:
dist_by_mode()

source,model,survey
tmodetp,,
Bike,5.74,3.03
HOV2,7.38,6.70
HOV3+,6.99,6.22
Park,16.33,24.39
SOV,8.94,8.50
TNC,2.55,3.99
Transit,10.37,9.40
Walk,0.95,0.89


## City of SeaTac

In [13]:
dist_by_mode(county_filter=1)

source,model,survey
tmodetp,,
Bike,6.38,2.00
HOV2,6.96,6.80
HOV3+,6.65,2.88
Park,12.31,13.50
SOV,7.75,7.88
TNC,2.83,NaN
Transit,10.83,9.28
Walk,1.08,0.50


## Other

In [14]:
dist_by_mode(county_filter=1, neg_county_filter=True)

source,model,survey
tmodetp,,
Bike,5.73,3.13
HOV2,7.38,6.70
HOV3+,7.00,6.23
Park,16.41,24.40
SOV,8.96,8.50
TNC,2.54,3.99
Transit,10.35,9.41
Walk,0.94,0.89


### Average Work Tour Distance By Mode

In [15]:
dist_by_mode('Work')

source,model,survey
tmodetp,,
Bike,6.11,4.31
HOV2,11.99,11.35
HOV3+,12.40,15.01
Park,16.33,23.26
SOV,12.57,11.78
TNC,6.29,4.86
Transit,14.42,10.77
Walk,0.76,1.29


## City of SeaTac

In [16]:
dist_by_mode('Work', 1)

source,model,survey
tmodetp,,
Bike,6.41,9.50
HOV2,9.80,5.53
HOV3+,9.76,7.50
Park,12.31,NaN
SOV,10.05,10.69
TNC,7.34,NaN
Transit,12.60,10.99
Walk,1.04,NaN


## Other

In [17]:
dist_by_mode('Work', 1,True)

source,model,survey
tmodetp,,
Bike,6.10,4.24
HOV2,12.04,11.53
HOV3+,12.46,15.01
Park,16.41,23.26
SOV,12.62,11.79
TNC,6.27,4.86
Transit,14.49,10.77
Walk,0.76,1.29


### Average School Tour Distance by Mode

In [18]:
dist_by_mode('School')

source,model,survey
tmodetp,,
Bike,2.79,1.30
HOV2,3.58,5.36
HOV3+,3.67,3.62
Park,NaN,28.46
SOV,7.38,6.08
School Bus,3.24,3.46
TNC,2.37,2.77
Transit,5.72,7.66
Walk,1.13,0.95


## City of SeaTac

In [19]:
dist_by_mode('School', 1)

source,model,survey
tmodetp,,
Bike,2.76,NaN
HOV2,3.63,3.40
HOV3+,3.80,3.50
SOV,7.66,NaN
School Bus,3.35,NaN
TNC,2.47,NaN
Transit,8.74,NaN
Walk,1.23,NaN


## Other

In [20]:
dist_by_mode('School', 1, True)

source,model,survey
tmodetp,,
Bike,2.79,1.30
HOV2,3.58,5.41
HOV3+,3.67,3.62
Park,NaN,28.46
SOV,7.37,6.08
School Bus,3.24,3.46
TNC,2.37,2.77
Transit,5.63,7.66
Walk,1.13,0.95


### Average Personal Business Tour Distance by Mode

In [21]:
dist_by_mode('Personal Business')

source,model,survey
tmodetp,,
Bike,3.56,3.05
HOV2,7.86,6.48
HOV3+,7.82,6.01
Park,NaN,19.28
SOV,6.64,5.33
TNC,1.99,4.93
Transit,6.87,6.09
Walk,1.19,0.90


## City of SeaTac

In [22]:
dist_by_mode('Personal Business', 1)

source,model,survey
tmodetp,,
Bike,4.69,1.50
HOV2,7.46,9.95
HOV3+,7.34,NaN
SOV,6.53,8.09
TNC,2.89,NaN
Transit,9.81,NaN
Walk,1.49,NaN


## Other

In [23]:
dist_by_mode('Personal Business', 1, True)

source,model,survey
tmodetp,,
Bike,3.54,4.70
HOV2,7.87,6.41
HOV3+,7.83,6.01
Park,NaN,19.28
SOV,6.65,5.33
TNC,1.98,4.93
Transit,6.76,6.09
Walk,1.19,0.90


### Average Shopping Tour Distance by Mode

In [24]:
dist_by_mode('Shop')

source,model,survey
tmodetp,,
Bike,2.62,0.56
HOV2,5.54,4.92
HOV3+,5.71,6.50
Park,NaN,0.50
SOV,4.76,3.76
TNC,1.43,2.64
Transit,4.38,2.26
Walk,1.08,0.63


## City of SeaTac

In [25]:
dist_by_mode('Shop', 1)

source,model,survey
tmodetp,,
Bike,3.52,NaN
HOV2,5.18,3.92
HOV3+,5.32,NaN
SOV,4.67,1.61
TNC,1.99,NaN
Transit,7.36,0.50
Walk,1.52,0.50


## Other

In [26]:
dist_by_mode('Shop', 1, True)

source,model,survey
tmodetp,,
Bike,2.61,0.56
HOV2,5.55,4.94
HOV3+,5.72,6.50
Park,NaN,0.50
SOV,4.76,3.78
TNC,1.42,2.64
Transit,4.28,2.38
Walk,1.07,0.63


### Average Meal Tour Distance by Mode

In [27]:
dist_by_mode('Meal')

source,model,survey
tmodetp,,
Bike,1.71,2.68
HOV2,8.78,4.59
HOV3+,9.40,6.29
Park,NaN,30.11
SOV,5.09,4.50
TNC,1.42,2.34
Transit,6.07,8.47
Walk,0.65,0.70


## City of SeaTac

In [28]:
dist_by_mode('Meal', 1)

source,model,survey
tmodetp,,
Bike,1.10,NaN
HOV2,8.85,1.50
HOV3+,9.49,NaN
SOV,5.10,3.49
TNC,1.45,NaN
Transit,9.15,NaN
Walk,0.65,NaN


## Other

In [29]:
dist_by_mode('Meal', 1, True)

source,model,survey
tmodetp,,
Bike,1.72,2.68
HOV2,8.78,4.59
HOV3+,9.40,6.29
Park,NaN,30.11
SOV,5.09,4.51
TNC,1.41,2.34
Transit,5.96,8.47
Walk,0.65,0.70


### Average Social Tour Distance by Mode

In [30]:
dist_by_mode('Social')

source,model,survey
tmodetp,,
Bike,2.80,2.33
HOV2,6.20,7.38
HOV3+,6.19,4.98
Park,NaN,54.15
SOV,5.52,5.65
TNC,1.43,3.02
Transit,5.20,11.23
Walk,0.97,0.88


## City of SeaTac

In [31]:
dist_by_mode('Social', 1)

source,model,survey
tmodetp,,
Bike,3.49,NaN
HOV2,6.38,7.17
HOV3+,6.16,2.30
Park,NaN,13.50
SOV,5.90,3.50
TNC,1.41,NaN
Transit,7.82,NaN
Walk,1.08,0.50


## Other

In [32]:
dist_by_mode('Social', 1, True)

source,model,survey
tmodetp,,
Bike,2.79,2.33
HOV2,6.20,7.39
HOV3+,6.19,5.01
Park,NaN,55.38
SOV,5.51,5.66
TNC,1.43,3.02
Transit,5.08,11.23
Walk,0.97,0.88


### Average Escort Tour Distance by Mode

In [33]:
dist_by_mode('Escort')

source,model,survey
tmodetp,,
Bike,13.61,0.65
HOV2,6.55,4.36
HOV3+,7.15,3.75
SOV,21.69,3.97
TNC,1.63,0.50
Transit,2.63,2.26
Walk,0.99,0.55


## City of SeaTac

In [34]:
dist_by_mode('Escort', 1)

source,model,survey
tmodetp,,
Bike,13.73,NaN
HOV2,6.54,14.50
HOV3+,7.04,NaN
SOV,14.88,NaN
TNC,1.81,NaN
Transit,2.50,NaN
Walk,1.00,NaN


## Other

In [35]:
dist_by_mode('Escort', 1, True)

source,model,survey
tmodetp,,
Bike,13.61,0.65
HOV2,6.55,4.27
HOV3+,7.15,3.75
SOV,21.80,3.97
TNC,1.63,0.50
Transit,2.64,2.26
Walk,0.99,0.55


Survey Sample Size Reference by Mode and Purpose

In [36]:
# Survey Sample size
df_tour_survey = pd.read_csv(r'inputs/base_year/survey/_tour.tsv', delim_whitespace=True)

In [37]:
df_tour_survey.pivot_table(index='tmodetp',columns='pdpurp', values='toexpfac', aggfunc='count')

pdpurp,1,2,3,4,5,6,7,10
tmodetp,,,,,,,,
1,889.00,189.00,158.00,375.00,631.00,811.00,"1,613.00",NaN
2,234.00,38.00,8.00,19.00,19.00,19.00,81.00,NaN
3,"3,215.00",114.00,43.00,890.00,820.00,352.00,937.00,1.00
4,945.00,284.00,637.00,560.00,587.00,373.00,743.00,NaN
5,455.00,442.00,477.00,336.00,287.00,246.00,659.00,1.00
6,"2,293.00",257.00,26.00,253.00,146.00,115.00,314.00,1.00
7,101.00,2.00,NaN,7.00,1.00,4.00,10.00,NaN
8,9.00,253.00,1.00,20.00,NaN,NaN,7.00,NaN
9,119.00,12.00,1.00,29.00,6.00,42.00,49.00,NaN


Survey Sample Size Reference by Mode and Purpose 
(for City of SeaTac)

In [38]:
df_household = pd.read_csv(r'inputs/base_year/survey/_household.tsv', delim_whitespace=True)
df_household = df_household[['hhno','hhtaz']]

seatac_tazs = [i for i in range(1, 212+1)]
df_household.loc[df_household['hhtaz'].isin(seatac_tazs), 'county'] = 'City of SeaTac'
df_household.loc[~df_household['hhtaz'].isin(seatac_tazs), 'county'] = 'Other'

In [39]:
index_map = {0:'Other', 1:'City of SeaTac'}
df_tour_survey = pd.merge(df_tour_survey, df_household, on="hhno", how='left')
for county, df_county in df_tour_survey.groupby('county'):
    print(county + ' \n')
    print(df_tour_survey.pivot_table(index=['tmodetp'],columns='pdpurp', values='toexpfac', aggfunc='count', fill_value=0))
    print('\n')

City of SeaTac 

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115   314   1
7         101    2    0    7    1    4    10   0
8           9  253    1   20    0    0     7   0
9         119   12    1   29    6   42    49   0
10        128   27    8   32   17   11    47   0


Other 

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115   3

# Work Tour Distance by Worker Type

In [40]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,12.53,12.15
1,Part-Time Worker,9.13,7.61
2,University Student,10.19,9.32


## City of SeaTac

In [41]:
df = load_data('tour_distance')
df = df.loc[df.person_subarea==1]
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,10.39,9.70
1,Part-Time Worker,8.89,6.74
2,University Student,9.41,NaN


## Other

In [42]:
df = load_data('tour_distance')
df = df.loc[df.person_subarea!=1]
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,12.57,12.17
1,Part-Time Worker,9.14,7.64
2,University Student,10.20,9.32
